In [6]:
from pathlib import Path
import sys
import os
import geopandas as gpd
import pandas as pd

scripts_dir = Path("..").joinpath("src")
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())
from mapshaper import Mapshaper
from tippcanoe import mbtileGeneration
from mapbox_uploader import uploadToMapbox
from pipelines.utils import load_regions

In [7]:
source_dir = Path("..").joinpath("data", "map_layer_sources").resolve()
file = source_dir.joinpath("./merged_mpa_all.json")
tileset = source_dir.joinpath("./merged_mpa_all.mbtiles")

In [3]:
Mapshaper(16).input([source_dir.joinpath("merged_mpa_all.shp").as_posix()]
                   ).filter_fields(fields="WDPAID,NAME,PA_DEF,REP_M_AREA,PARENT_ISO"
                                  ).clean(allow_overlaps=True, rewind=True).simplify(
                                        "dp 10% keep-shapes planar"
                                    ).clean(allow_overlaps=True).output(
                                        source_dir.joinpath("merged_mpa_all_simp.json").as_posix(), force=True, format="geojson"
).debug()

mapshaper-xl 16gb -i /home/mambauser/data/map_layer_sources/merged_mpa_all.shp  -filter-fields fields=WDPAID,NAME,PA_DEF,REP_M_AREA,PARENT_ISO -clean allow-overlaps rewind -simplify dp 10% keep-shapes planar -clean allow-overlaps -o /home/mambauser/data/map_layer_sources/merged_mpa_all_simp.json force format=geojson


In [4]:
mbtileGeneration(file)

FileNotFoundError: Data path does not exist.

In [ ]:
uploadToMapbox(
    tileset,
    "mpas_wdpa",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/map_layer_sources/merged_mpa_all.mbtiles to s3://tilestream-tilesets-production/71/_pending/rgtjc2em7tn1bcm24tqf2yolc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [05:40<00:00,  3.41s/it]


True

In [ ]:
regions_df = pd.DataFrame(
    [
        {"region_id": data["region_iso"], "location_id": iso}
        for data in load_regions().get("data", [])
        for iso in data["country_iso_3s"]
    ]
)

In [ ]:
eez_dir = Path("..").joinpath("data", "eez_intermediate", "eez_intermediate").resolve()
eez_data = gpd.read_file(eez_dir.joinpath("eez_intermediate.shp").as_posix())

In [ ]:
gpd.GeoDataFrame(
    pd.merge(
        eez_data,
        regions_df,
        how="left",
        left_on="ISO_SOV1",
        right_on="location_id",
        sort=True,
        copy=True,
    ),
    crs=eez_data.crs,
).to_file(filename=eez_dir.joinpath("eez_intermediate_regions.shp").as_posix(), driver="ESRI Shapefile")

/tmp/ipykernel_431/274243943.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(


In [ ]:
Mapshaper(16).input([eez_dir.joinpath("eez_intermediate_regions.shp").as_posix()]).dissolve2(
    fields="region_id"
).output(eez_dir.joinpath("regions.json").as_posix(), force=True, format="geojson").execute()

Allocating 16 GB of heap memory
[dissolve2] Dissolved 282 features into 8 features
[o] Wrote /home/mambauser/data/eez_intermediate/eez_intermediate/regions.json


CompletedProcess(args='mapshaper-xl 16gb -i /home/mambauser/data/eez_intermediate/eez_intermediate/eez_intermediate_regions.shp  -dissolve2 fields=region_id -o /home/mambauser/data/eez_intermediate/eez_intermediate/regions.json force format=geojson', returncode=0)

In [ ]:
mbtileGeneration(eez_dir.joinpath("regions.json"))

For layer 0, using name "regions"
/home/mambauser/data/eez_intermediate/eez_intermediate/regions.json:2: Found ] at top level
/home/mambauser/data/eez_intermediate/eez_intermediate/regions.json:3: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
8 features, 60659758 bytes of geometry, 40 bytes of separate metadata, 85 bytes of string pool
Choosing a maxzoom of -z0 for features about 26898822 feet (8198761 meters) apart
Choosing a maxzoom of -z9 for resolution of about 901 feet (274 meters) within features
  99.9%  9/95/295   


PosixPath('/home/mambauser/data/eez_intermediate/eez_intermediate/regions.mbtiles')

In [ ]:
uploadToMapbox(
    eez_dir.joinpath("regions.mbtiles"),
    "regions",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/eez_intermediate/eez_intermediate/regions.mbtiles to s3://tilestream-tilesets-production/57/_pending/bs3219kj1ot1elh00sc95yolc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [03:05<00:00,  1.86s/it]


True

In [12]:
mpa_atlas = source_dir.parent.joinpath("mpaatlas_intermediate", "mpaatlas_intermediate")
mpa_atlas

PosixPath('/home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate')

In [13]:
mbtileGeneration(mpa_atlas.joinpath("mpaatlas_intermediate.shp"))

Allocating 50 GB of heap memory
[clean] Retained 511 of 524 features
[o] Wrote /home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.json
For layer 0, using name "mpaatlas_intermediate"
/home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.json:43: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
/home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.json:20: Found ] at top level
511 features, 32444137 bytes of geometry, 11240 bytes of separate metadata, 27786 bytes of string pool
Choosing a maxzoom of -z0 for features about 521726 feet (159022 meters) apart
Choosing a maxzoom of -z12 for resolution of about 97 feet (29 meters) within features
  99.9%  12/1378/3481  


PosixPath('/home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.mbtiles')

In [14]:
uploadToMapbox(
    mpa_atlas.joinpath("mpaatlas_intermediate.mbtiles"),
    "mpa_atlas",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.mbtiles to s3://tilestream-tilesets-production/43/_pending/23ii4osj9qn1ppy2c6aehzolc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [05:51<00:00,  3.51s/it]


True

In [15]:
protectedSeas = Path("..").joinpath("data", "ProtectedSeas").resolve()

In [16]:
Mapshaper(16).input([protectedSeas.joinpath("protectedseas.shp").as_posix()]).clean(
    allow_overlaps=True, rewind=True
).simplify("dp 10% keep-shapes planar").clean(allow_overlaps=True).output(
    protectedSeas.joinpath("protectedseas.json").as_posix(), force=True, format="geojson"
).execute()

Allocating 16 GB of heap memory
[clean] Retained 10,879 of 10,879 features
[simplify] Repaired 110,498 intersections; 22,095 intersections could not be repaired
[clean] Retained 10,879 of 10,879 features
[o] Wrote /home/mambauser/data/ProtectedSeas/protectedseas.json


CompletedProcess(args='mapshaper-xl 16gb -i /home/mambauser/data/ProtectedSeas/protectedseas.shp  -clean allow-overlaps rewind -simplify dp 10% keep-shapes planar -clean allow-overlaps -o /home/mambauser/data/ProtectedSeas/protectedseas.json force format=geojson', returncode=0)

In [17]:
mbtileGeneration(protectedSeas.joinpath("protectedseas.json"))

For layer 0, using name "protectedseas"
/home/mambauser/data/ProtectedSeas/protectedseas.json:967: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
/home/mambauser/data/ProtectedSeas/protectedseas.json:378: Found ] at top level
10879 features, 31016278 bytes of geometry, 377536 bytes of separate metadata, 766016 bytes of string pool
Choosing a maxzoom of -z3 for features about 58402 feet (17801 meters) apart
Choosing a maxzoom of -z11 for resolution of about 245 feet (74 meters) within features
  99.9%  11/183/597    


PosixPath('/home/mambauser/data/ProtectedSeas/protectedseas.mbtiles')

In [18]:
uploadToMapbox(
    protectedSeas.joinpath("protectedseas.mbtiles"),
    "protected_seas",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/ProtectedSeas/protectedseas.mbtiles to s3://tilestream-tilesets-production/86/_pending/uga5xrvzspo15013gnpttzolc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [03:52<00:00,  2.32s/it]


True